In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
import tqdm.notebook as notebook

import tensorflow as tf


BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
import utils.custom_tf

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X_SIZE = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y_SIZE = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y_SIZE, IMAGE_X_SIZE)

print(DATASET, RANDOM_SEED)

# folders for processed, models
DATA_F = os.path.join(BASE_DIR, f"data/{DATASET}/")
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}')

BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base") # mt = model_type

# model saved here
if os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: model has been made for rs={RANDOM_SEED}")

adience 15


In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "train"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

hyper_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "hyper_train"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

hyper_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "hyper_val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

Found 9795 files belonging to 2 classes.
Found 2449 files belonging to 2 classes.
Found 399 files belonging to 2 classes.
Found 399 files belonging to 2 classes.
Found 3585 files belonging to 2 classes.


In [5]:
'''
This will standardize the pixel data
'''
def preprocess(imgs, labels):
    # turn from <0..255> to <0..1>
    imgs = imgs / 255.0
    means = np.array( [0.5, 0.5, 0.5] )
    stds = np.array( [0.5, 0.5, 0.5] )
    imgs = (imgs - means) / stds
    return imgs, labels

In [6]:
# after loading the data, this will efficiently preprocess it in real-time
# this approach is 3x faster than `flow_from_directory`
train_ds = train_ds.map(preprocess)
hyper_train_ds = hyper_train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
hyper_val_ds = hyper_val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

val_full_ds = val_ds.concatenate(hyper_val_ds)

In [7]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=2,
)

In [8]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [9]:
optimizer = tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9)

In [10]:
model.compile(optimizer, loss='categorical_crossentropy', metrics = ['accuracy'])

In [11]:
# this will periodically show the accuracy on val and test
show_valid_test = utils.custom_tf.ShowValidAndTest(
    val_full_ds,
    test_ds,
    epoch_freq=5,
)

def scheduler(epoch):
    if epoch > 10:
        return 5e-5
    else:
        return 1e-4

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=BASE_MODEL_SAVEPATH,
    monitor="val_loss",
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

callbacks = [show_valid_test, lr_scheduler, save_best]

In [13]:
history = model.fit(
        x=train_ds,
        epochs=EPOCHS,
        validation_data=hyper_train_ds,
        verbose=1,
        callbacks=callbacks,
)

In [14]:
# load best model
model.load_weights(BASE_MODEL_SAVEPATH)

In [14]:
# hyper train acc
preds, labels = utils.utils.compute_preds(
    model,
    hyper_train_ds,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds, axis=1) == labels).mean()

100%|██████████| 77/77 [00:17<00:00,  4.52it/s]


0.8035933033891384

In [16]:
# val acc
preds, labels = utils.utils.compute_preds(
    model,
    val_full_ds,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds, axis=1) == labels).mean()

100%|██████████| 13/13 [00:02<00:00,  4.81it/s]


0.7738693467336684

In [17]:
# test acc
preds, labels = utils.utils.compute_preds(
    model,
    test_ds,
    batch_size=BATCH_SIZE,
)
(np.argmax(preds, axis=1) == labels).mean()

100%|██████████| 100/100 [00:20<00:00,  4.84it/s]


0.7762786319422654